In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt

In [2]:
# Read ny_grocery.csv into a DataFrame: ny_grocery
ny_grocery = pd.read_csv('ny_grocery.csv')

# Print the head of ny_grocery
print(ny_grocery.head())

                    street  zip_code                brand        LAT  \
0     2073 Broadway, 10023   10023.0  Trader Joe's & Aldi  40.778582   
1       675 6th Ave, 10010   10010.0  Trader Joe's & Aldi  40.741878   
2  436 East 14th St, 10009   10009.0  Trader Joe's & Aldi  40.730509   
3   400 Grand St. (Cellar)   10002.0  Trader Joe's & Aldi  40.716159   
4            200 E 32nd St   10016.0  Trader Joe's & Aldi  40.744028   

         LON  
0 -73.982257  
1 -73.993834  
2 -73.981099  
3 -73.986726  
4 -73.979059  


In [3]:
# Read ny_grocery.csv into a DataFrame: ny_grocery
n_df = pd.read_csv('2020_Neighborhood_Tabulation_Areas__NTAs__-_Tabular.csv')

# Print the head of ny_grocery
print(n_df.head())

                                            the_geom  BoroCode  BoroName  \
0  MULTIPOLYGON (((-73.93213397515774 40.72815960...         3  Brooklyn   
1  MULTIPOLYGON (((-73.96176070350778 40.72522879...         3  Brooklyn   
2  MULTIPOLYGON (((-73.95023693757913 40.70547324...         3  Brooklyn   
3  MULTIPOLYGON (((-73.92405909736993 40.71411156...         3  Brooklyn   
4  MULTIPOLYGON (((-73.9923636697722 40.689690123...         3  Brooklyn   

   CountyFIPS NTA2020             NTAName  NTAAbbrev  NTAType CDTA2020  \
0          47  BK0101          Greenpoint      Grnpt        0     BK01   
1          47  BK0102        Williamsburg   Wllmsbrg        0     BK01   
2          47  BK0103  South Williamsburg  SWllmsbrg        0     BK01   
3          47  BK0104   East Williamsburg  EWllmsbrg        0     BK01   
4          47  BK0201    Brooklyn Heights      BkHts        0     BK02   

                                            CDTAName    Shape_Leng  \
0     BK01 Williamsburg-Gree

In [4]:
# Convert the 'the_geom' column in the neighborhoods dataframe from WKT to geometries
n_df['the_geom'] = n_df['the_geom'].apply(wkt.loads)
n_gdf = gpd.GeoDataFrame(n_df, geometry='the_geom')

In [5]:
# Create a GeoDataFrame for stores using the LAT and LON columns for geometry
gdf = gpd.GeoDataFrame(ny_grocery, geometry=gpd.points_from_xy(ny_grocery.LON, ny_grocery.LAT))

# Ensure both GeoDataFrames use the same CRS
# It's common to use WGS84 (epsg:4326) for geographic coordinates
n_gdf.crs = "EPSG:4326"
gdf.crs = "EPSG:4326"

# Perform spatial join: this assigns each store to a neighborhood
joined = gpd.sjoin(gdf, n_gdf, how="inner", op="within")

/Users/thomaslee/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [6]:
# Count the number of stores in each neighborhood
store_counts = joined['NTAName'].value_counts()

# Convert the counts to a DataFrame
store_counts_df = store_counts.reset_index()
store_counts_df.columns = ['NTAName', 'StoreCount']

print(store_counts_df)

                                          NTAName  StoreCount
0              University Heights (North)-Fordham           9
1                                    Forest Hills           8
2    Carroll Gardens-Cobble Hill-Gowanus-Red Hook           8
3                                        Flatbush           7
4              Claremont Village-Claremont (East)           7
..                                            ...         ...
162                           South Richmond Hill           1
163                  Oakland Gardens-Hollis Hills           1
164                                       Midwood           1
165                                     Bellerose           1
166                        New Dorp-Midland Beach           1

[167 rows x 2 columns]


In [7]:
#Save the store_counts_df to a csv file
store_counts_df.to_csv('store_counts.csv', index=False)